# Generate Conversations by Prompt Engineering

This notebook generates natural conversations between a patient and a healthcare chatbot. In the conversations, the patient describes their daily activities to the chatbot, and the chatbot asks further questions for more detailed information.

This step utilises code from the following GitHub repository:

[ICF-activities-classifier](https: //github.com/cltl-students/ICF-activities-classifier)

In [1]:
import os
os.environ["OPENAI_API_KEY"] = '' # Enter OpenAI API key

In [2]:
import openai
import os
import IPython
import time
import json
# from dotenv import load_dotenv
# load_dotenv()


In [3]:
# Check the OpenAI API key is set
print("OpenAI API Key: {}".format(openai.api_key))

OpenAI API Key: 


In [4]:
# Define the categories under the ICF framework
ICF_defs = {
    "mobility": "about moving by changing body position or location or by transferring from one place to another, by carrying, moving or manipulating objects, by walking, running or climbing, and by using various forms of transportation",

    "self-care": "about caring for oneself, washing and drying oneself, caring for one's body and body parts, dressing, eating and drinking, and looking after one's health",

    "domestic life": "about carrying out domestic and everyday actions and tasks. Areas of domestic life include caring for one's belongings and space, acquiring food, clothing and other necessities, household cleaning and repairing, caring for personal and other household objects, and assisting others"
            }

In [6]:
# Define the specific activities under each category
allevents = {
    "mobility": {
        "changing and maintaining body position": "getting up from a chair to lie down on a bed, getting into or out of sitting, standing, kneeling or squatting, maintaining seated, remaining standing and transfering oneself",

        "carrying, moving and handling objects": "lifting, carrying, kicking a ball, pushing bicycle pedal, using one's hands, fingers and thumbs, using one's hands and arms, and using one's foot and toes",

        "walking and moving": "walking, going up stairs, going down stairs, skipping, sacmpering, jummping, someraulting, skating, skiing, using a self-propelled wheelchair and walking with a walker",

        "moving around using transportation": "taking a taxi, bus, tram, train, subway, boat, airplane or a car, driving a car or a boat, riding a bike and riding a horse or a camel",
    },

    "self-care": {
        "washing oneself": "bathing, showering, washing hands, washing feets, washing face, washing hair, drying body with a towel",

        "caring for body":"doing skincare, brushing teeth, cutting nails and doing genital care",

        "toileting": "cleaning menstruation, peeing, pooping, and cleaning oneself after toileting",

        "dressing": "putting on clothings and taking off clothings",

        "eating": "having meals, cutting food into pieces and using cutleries",

        "drinking": "drinking, stirring liquids, pouring liquids, opening bottles and drinking with a straw",

        "looking after one's health": "maintaining a balanced diet, keeping warm or cool and excercising",
    },

    "domestic life": {
        "acquisition of necessities":"buying dewellings, renting dewellings, furnishing dewellings, arranging dewellings, selecting household goods, buying household goods, transporting household goods and acquring household services",

        "household tasks": "preparing meals, making a menu, cooking meals, serving food, doing housework, cleaning the house, washing laundry, using household appliances, storing food and disposing garbage",

        "caring for household objects and assisting others": "maintaining household, repairing household, taking care of plants or animals and assisting household members",
    },
            }

In [7]:
ICF_defs.values()
allevents.values()

dict_values([{'changing and maintaining body position': 'getting up from a chair to lie down on a bed, getting into or out of sitting, standing, kneeling or squatting, maintaining seated, remaining standing and transfering oneself', 'carrying, moving and handling objects': "lifting, carrying, kicking a ball, pushing bicycle pedal, using one's hands, fingers and thumbs, using one's hands and arms, and using one's foot and toes", 'walking and moving': 'walking, going up stairs, going down stairs, skipping, sacmpering, jummping, someraulting, skating, skiing, using a self-propelled wheelchair and walking with a walker', 'moving around using transportation': 'taking a taxi, bus, tram, train, subway, boat, airplane or a car, driving a car or a boat, riding a bike and riding a horse or a camel'}, {'washing oneself': 'bathing, showering, washing hands, washing feets, washing face, washing hair, drying body with a towel', 'caring for body': 'doing skincare, brushing teeth, cutting nails and do

In [16]:
def category_testset(category, definition, allevents, MODEL = "gpt-3.5-turbo", dir= '../response_data/raw_conversations/'):
    """
    This is to generate dialogues acccording to categories in ICF by feeding the definitions of the each category to the prompt
    """

    if not os.path.exists(dir):
        os.makedirs(dir)
    
    count = 0
    data = {f"{category}": []}  
    text2write = ""
    for subcategory, events in allevents[category].items():

        query = [
            {"role": "system", "content": "Think as you are a real human."}, # Human
            {"role": "system", "content": """Generate one small and natural
            conversation without greetings by playing the roles of a friend (F) 
            and an elderly patient (P). The conversation has 6 utterances, 
            mentioning one or two events. Each utterance should be completed
            and has less than 20 tokens. The format is as below:
            F: utterance
            P: utternace
            F: utterance
            ...
            """}, # conversation format discription roles
            
            {"role": "user", "content": f"The topic of the conversation is 
            about {category} events. In terms of functioning, {category} is 
            about {definition}. {category} events include {events}."}, # shifting details in the conversations
            
            {"role": "user", "content": f"The patient can talk with the 
            friend about his/her daily life which can reflect his/her 
            functioning in {category}. The friend should ask when and 
            how the event occurs. "}# descripe category with definitions (listing events)
            ]
        
        num_60 = ["self-care"]
        num_70 = ["mobility"]
        num_80 = ["domestic life"]

        if category in num_60:
            num = 60
        elif category in num_70:
            num = 70
        elif category in num_80:
            num = 80

        else:
            print(f"no such category: {category}")

        print(f"gpt-runing {category} - {subcategory}")
        # MODEL = "gpt-4o"
        response_query = openai.ChatCompletion.create(
        model=MODEL,
        messages=query,
        temperature=1.5,
        max_tokens = 150,
        n = num, 
        )

        print(f"gpt finished {category} - {subcategory}")
        
        # IPython.display.Markdown(response_query.choices[0]['message']['content'])
        time.sleep(60)
    
        for text in response_query.choices:
            conversation = text['message']['content']
            one_conversation = conversation + '\n'+'-'*10 +'\n'
            data[f"{category}"].append(conversation)
            text2write += one_conversation
            count += 1

    with open (f'{dir}{category}.txt','w') as tfile:
        tfile.write(text2write)

    with open (f'{dir}{category}.json','w', encoding='utf-8') as jfile:
            json.dump(data,jfile, ensure_ascii=False, indent=4)

    print(f"generated {count} {category} conversations")
    print("-"*30)
    print("\n")


In [15]:
for category, definition, in ICF_defs.items():
    category_testset(category, definition, allevents)
print("done")

gpt-runing mobility - changing and maintaining body position
gpt finished mobility - changing and maintaining body position
gpt-runing mobility - carrying, moving and handling objects
gpt finished mobility - carrying, moving and handling objects
gpt-runing mobility - walking and moving
gpt finished mobility - walking and moving
gpt-runing mobility - moving around using transportation
gpt finished mobility - moving around using transportation
generated 280 mobility conversations
------------------------------


gpt-runing self-care - washing oneself
gpt finished self-care - washing oneself
gpt-runing self-care - caring for body
gpt finished self-care - caring for body
gpt-runing self-care - toileting
gpt finished self-care - toileting
gpt-runing self-care - dressing
gpt finished self-care - dressing
gpt-runing self-care - eating
gpt finished self-care - eating
gpt-runing self-care - drinking
gpt finished self-care - drinking
gpt-runing self-care - looking after one's health
gpt finished